<a href="https://colab.research.google.com/github/cicattzo/nlp_project/blob/main/6_684_project_baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install sumy

  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=d5972628af9a677e41f16cb0508c32b8f79fe0f0176553fdf81bd3eca1c495a6
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21680 sha256=5d36dc7c46e801cf7bdf5449fd37c1c24c9e83041e310225999417bc9c1f653d
  Stored in directory: /root/.cache/pip/wheels/5a/4d/a1/510b12c5e65e0b2b3ce539b2af66da0fc57571e528924f4a52
Successfully built pycountry breadability


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
import pandas as pd
import numpy as np
import re
# import xml.etree.ElementTree as ET
# from talon.signature.bruteforce import extract_signature
# import email

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Basic structure for summarization with Sumy

In [ ]:
LANGUAGE = "english"
SENTENCES_COUNT = 10
tokenizer = Tokenizer(LANGUAGE)
stemmer = Stemmer(LANGUAGE)
# change this line to any other summarizer
summarizer = TextRankSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

loading pre-processed BC3 emails corpus

In [ ]:
bc3_df = pd.read_csv("/content/gdrive/My Drive/6864_project/bc3_processed.csv")

In [ ]:
txt = bc3_df.iloc[0]['body']
summary = bc3_df.iloc[0]['summary']

In [ ]:
parser = PlaintextParser.from_string(txt, tokenizer)

In [ ]:
print("The Text:\n")
print(txt)
print("\nThe Summary:\n")
print(summary)
print("\nText Rank\n")

for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)
    break

The Text:

The IETF meetings tend to become too large, creating logistics and planning problems. I suggest that future meetings are held for two weeks, with applications and user services issues the first week, and all other issues the second week. Those who so wish could attend both weeks, and other people could attend only one week. Those who choose to attend both weeks would be able to cover more groups and do better liaisons between the different areas. The Friday of the first week could discuss applications issues which might be of special interest to the other areas, and the Monday of the second week would schedule other groups which might be of special interest to applications people, so some people could attend Monday-Monday or Friday-Friday. 
------------------------------------------------------------------------ 
Jacob Palme &lt;jpalme@dsv.su.se&gt; (Stockholm University and KTH) 
for more info see URL: http://www.dsv.su.se/~jpalme

The Summary:

Jacob suggested to hold two 

In [ ]:
bc3_df.iloc[0]['subject']

'Extending IETF meetings to two weeks?'

In [ ]:
bc3_df['listno'] + "-" + bc3_df['email_num'].astype(str)

0       007-7484738-1
1       007-7484738-1
2       007-7484738-1
3       007-7484738-2
4       007-7484738-2
            ...      
664    107-16164699-6
665    107-16164699-6
666    107-16164699-6
667    107-16164699-7
668    107-16164699-7
Length: 669, dtype: object

Converting BC3 corpus to a dict

In [ ]:
# keeping the subject and body separate, but they can be merged
bc3_df['unique_key'] = bc3_df['listno'] + "-" + bc3_df['email_num'].astype(str)
bc3_dict = bc3_df.groupby('unique_key').agg({'subject':lambda x: x.iloc[0], 'body':lambda x: x.iloc[0], 'summary':lambda x: x.to_list()}).to_dict()

The structure of the dict is: \
{ \
  'subject': Dict(str: unique_key => str: subject_text) \
  'body':    Dict(str: unique_key => str: body_text) \
  'summary': Dict(str: unique_key => list: candidate_summaries ) \
  } \\

  Look at the example below for more clarity
  

In [ ]:
keys = list(bc3_dict['summary'].keys())
# candidate summaries for the first email
bc3_dict['summary'][keys[0]]

['Jacob suggested to hold two week meetings, the first will discuss user service issues and all other issues the following week, he said people can attend one week or both.',
 'Jacob suggests that future IETF meetings be spread over two weeks.',
 'The topic is the logistics of scheduling IETF meetings A suggestion is made to hold future meetings for two weeks to spread out the volume of participants']